In [1]:
from utils import *
from dataloader import *
from TSRNet import TSRNet
import numpy as np
import torch

import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


import scipy.signal
from scipy.signal import find_peaks


In [4]:
# Custom collate function to handle 'when' data
def custom_collate_fn(batch):
    time_instance, spec_instance, timestamp = zip(*batch)
    time_instance = torch.stack(time_instance)
    spec_instance = torch.stack(spec_instance)
    timestamp = np.array(timestamp, dtype=object)
    return time_instance, spec_instance, timestamp

# 폴더 경로 설정
test_folder_path = 'C:/Users/SKTPJ15/real/valdata'
processed_test_data = process_all_csv_files(test_folder_path)
test_dataset = Testset(processed_test_data)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn)

abnormal
Processing file: C:/Users/SKTPJ15/real/multimodal/dd\whyrae.csv


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pt_path = "C:/Users/SKTPJ15/real/multimodal/new_save0/TSRNet-63.pt"

model = TSRNet(enc_in=3).to(device)
checkpoint = torch.load(pt_path,  weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [6]:
threshold = 0.04

def test_with_anomaly_detection(model, test_loader, threshold, save_path):
    model.eval()
    anomalies_detected = False
    reconstruction_errors = []
    
    
    for i, (time_instance, spec_instance, timestamp) in tqdm(enumerate(test_loader)):
        with torch.no_grad():
            time_instance = time_instance.float().to(device)
            spec_instance = spec_instance.float().to(device)
            
            (gen_time, time_var) = model(time_instance, spec_instance)
            
            time_err = (gen_time - time_instance) ** 2
            reconstruction_error = torch.mean(time_err).item()
            reconstruction_errors.append(reconstruction_error)

            if reconstruction_error > threshold:
                
                anomalies_detected = True
                file_name_0 = timestamp[0, 0, 0]
                chunk_index = timestamp[0, 0, 1]
                
                print(f"Anomaly detected in batch {i}, file {file_name_0}, chunk {chunk_index}")
                print(f"Reconstruction Error: {reconstruction_error:.6f}")
                    
                timestamp_start = timestamp[0, 0, 2]
                timestamp_end = timestamp[0, -1, 2]
                timestamp_start = datetime.datetime.fromtimestamp(float(timestamp_start) / 1000.0)
                timestamp_end = datetime.datetime.fromtimestamp(float(timestamp_end) / 1000.0)
                formatted_start = timestamp_start.strftime('%Y-%m-%d %H:%M:%S')
                formatted_end = timestamp_end.strftime('%Y-%m-%d %H:%M:%S')
                print(f"Start timestamp: {formatted_start}      End timestamp: {formatted_end}")
                print()
                
    if not anomalies_detected:
        print("No anomalies detected.")
        
    # Save the reconstruction errors to a CSV file
    df = pd.DataFrame(reconstruction_errors, columns=['Reconstruction Error'])
    df.to_csv(save_path, index=False)
    print(f"Reconstruction errors saved to {save_path}")


save_path = "63_abnormal_reconstruction_errors.csv"
# Assume test_loader is already defined and loaded with your test data
test_with_anomaly_detection(model, test_loader, threshold, save_path)

33it [00:01, 23.69it/s]

Anomaly detected in batch 0, file whyrae.csv, chunk 0
Reconstruction Error: 0.015784
Start timestamp: 381403      End timestamp: 381962

Anomaly detected in batch 1, file whyrae.csv, chunk 1
Reconstruction Error: 0.015784
Start timestamp: 381963      End timestamp: 382522

Anomaly detected in batch 2, file whyrae.csv, chunk 2
Reconstruction Error: 0.015784
Start timestamp: 382523      End timestamp: 383082

Anomaly detected in batch 3, file whyrae.csv, chunk 3
Reconstruction Error: 0.015784
Start timestamp: 383083      End timestamp: 383642

Anomaly detected in batch 4, file whyrae.csv, chunk 4
Reconstruction Error: 0.015784
Start timestamp: 383643      End timestamp: 384202

Anomaly detected in batch 5, file whyrae.csv, chunk 5
Reconstruction Error: 0.015784
Start timestamp: 384203      End timestamp: 384762

Anomaly detected in batch 6, file whyrae.csv, chunk 6
Reconstruction Error: 0.015784
Start timestamp: 384763      End timestamp: 385322

Anomaly detected in batch 7, file whyrae.

97it [00:02, 81.85it/s]

Anomaly detected in batch 64, file whyrae.csv, chunk 64
Reconstruction Error: 0.015784
Start timestamp: 417243      End timestamp: 417802

Anomaly detected in batch 65, file whyrae.csv, chunk 65
Reconstruction Error: 0.015784
Start timestamp: 417803      End timestamp: 418362

Anomaly detected in batch 66, file whyrae.csv, chunk 66
Reconstruction Error: 0.015784
Start timestamp: 418363      End timestamp: 418922

Anomaly detected in batch 67, file whyrae.csv, chunk 67
Reconstruction Error: 0.015784
Start timestamp: 418923      End timestamp: 419482

Anomaly detected in batch 68, file whyrae.csv, chunk 68
Reconstruction Error: 0.015784
Start timestamp: 419483      End timestamp: 420042

Anomaly detected in batch 69, file whyrae.csv, chunk 69
Reconstruction Error: 0.015784
Start timestamp: 420043      End timestamp: 420602

Anomaly detected in batch 70, file whyrae.csv, chunk 70
Reconstruction Error: 0.015784
Start timestamp: 420603      End timestamp: 421162

Anomaly detected in batch 7

158it [00:02, 143.64it/s]

...

Anomaly detected in batch 297, file whyrae.csv, chunk 297
Reconstruction Error: 0.015784
Start timestamp: 769546      End timestamp: 770105

Anomaly detected in batch 298, file whyrae.csv, chunk 298
Reconstruction Error: 0.015784
Start timestamp: 770106      End timestamp: 770665

Anomaly detected in batch 299, file whyrae.csv, chunk 299
Reconstruction Error: 0.015784
Start timestamp: 770666      End timestamp: 771225

Anomaly detected in batch 300, file whyrae.csv, chunk 300
Reconstruction Error: 0.015784
Start timestamp: 771226      End timestamp: 771785

Anomaly detected in batch 301, file whyrae.csv, chunk 301
Reconstruction Error: 0.015784
Start timestamp: 771786      End timestamp: 772345

Anomaly detected in batch 302, file whyrae.csv, chunk 302
Reconstruction Error: 0.015784
Start timestamp: 772346      End timestamp: 772905

Anomaly detected in batch 303, file whyrae.csv, chunk 303
Reconstruction Error: 0.015784
Start timestamp: 772906      End timestamp: 773465

Anomaly detec

408it [00:03, 270.02it/s]

Anomaly detected in batch 358, file whyrae.csv, chunk 358
Reconstruction Error: 0.015784
Start timestamp: 803706      End timestamp: 804265

Anomaly detected in batch 359, file whyrae.csv, chunk 359
Reconstruction Error: 0.015784
Start timestamp: 804266      End timestamp: 804825

Anomaly detected in batch 360, file whyrae.csv, chunk 360
Reconstruction Error: 0.015784
Start timestamp: 804826      End timestamp: 805385

Anomaly detected in batch 361, file whyrae.csv, chunk 361
Reconstruction Error: 0.015784
Start timestamp: 805386      End timestamp: 805945

Anomaly detected in batch 362, file whyrae.csv, chunk 362
Reconstruction Error: 0.015784
Start timestamp: 805946      End timestamp: 806505

Anomaly detected in batch 363, file whyrae.csv, chunk 363
Reconstruction Error: 0.015784
Start timestamp: 806506      End timestamp: 807065

Anomaly detected in batch 364, file whyrae.csv, chunk 364
Reconstruction Error: 0.015784
Start timestamp: 807066      End timestamp: 807625

Anomaly detec

447it [00:03, 131.56it/s]

Anomaly detected in batch 414, file whyrae.csv, chunk 414
Reconstruction Error: 0.015784
Start timestamp: 835066      End timestamp: 835625

Anomaly detected in batch 415, file whyrae.csv, chunk 415
Reconstruction Error: 0.015784
Start timestamp: 835626      End timestamp: 836185

Anomaly detected in batch 416, file whyrae.csv, chunk 416
Reconstruction Error: 0.015784
Start timestamp: 836186      End timestamp: 836745

Anomaly detected in batch 417, file whyrae.csv, chunk 417
Reconstruction Error: 0.015784
Start timestamp: 836746      End timestamp: 837305

Anomaly detected in batch 418, file whyrae.csv, chunk 418
Reconstruction Error: 0.015784
Start timestamp: 837306      End timestamp: 837865

Anomaly detected in batch 419, file whyrae.csv, chunk 419
Reconstruction Error: 0.015784
Start timestamp: 837866      End timestamp: 838425

Anomaly detected in batch 420, file whyrae.csv, chunk 420
Reconstruction Error: 0.015784
Start timestamp: 838426      End timestamp: 838985

Anomaly detec